# Text Generation

It is now time to generate some text from the models we trained!

As a recap:
 - we trained a **first bidirectional LSTM model** to predict the next word of a given sequence of 30 words.
 - we train a **doc2vec model** for the whole input text as space, sentence based,
 - we trained a **second bidirectional LSTM model** to predict the best vectorized-sentence, following a sequence of 15 vectorized-phrases.
 
So, what will be the process of our text generation ? We have first to provide a seed of 15 sentences, that contain at least 30 words. then:
 1. using the last 30 words of the seed, we generate 10 candidates sentences.
 2. we infer their vectors using the doc2vec model,
 3. we calculate the "best vector" for the sentence following the 15 phrases of the seed,
 4. we compare the infered vectors with the "best vector", and pick-up the closest one.
 5. we add the generated sentence corresponding to this vector at the end of the seed, as the next sentence of the text.
 6. then, we loop over the process.
 
## 0. import libraries and parameters

In order to start, we have to import our models and retrieve our vocabulary.

In [1]:
from __future__ import print_function
import double_log
def print(*args, **kwargs):
    return double_log.print(*args, **kwargs)

import numpy as np
import os
import scipy
from six.moves import cPickle

In [2]:
save_dir = 'save' # directory to store models

In [3]:
#import spacy, and french model
import spacy
nlp = spacy.load('en')

In [4]:
#import gensim library
import gensim
from gensim.models.doc2vec import LabeledSentence

#load the doc2vec model
print("loading doc2Vec model...")
d2v_model = gensim.models.doc2vec.Doc2Vec.load('./data/doc2vec.w2v')

print("model loaded!")

loading doc2Vec model...
model loaded!


In [5]:
#load vocabulary
print("loading vocabulary...")
vocab_file = os.path.join(save_dir, "words_vocab.pkl")

with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
        words, vocab, vocabulary_inv = cPickle.load(f)

vocab_size = len(words)
print("vocabulary loaded !")

loading vocabulary...
vocabulary loaded !


In [6]:
from keras.models import load_model
import random
from os import listdir
from os.path import isfile, join
# load the keras models
print("loading word prediction model...")

model = load_model(save_dir + "/" + 'my_model_gen_sentences_lstm.final.hdf5')
print("model loaded!")
print("loading sentence selection model...")
model_sequence = load_model(save_dir + "/" + 'my_model_sequence_lstm.final2.hdf5')
print("model loaded!")

/home/nyuad/anaconda2/envs/Gabor/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


loading word prediction model...
model loaded!
loading sentence selection model...
model loaded!


# 1. Functions to generate Candidates Sentences


To improve the text generation, and tune a bit the word prediction, we introduce a specific function to pick-up words from our vocabulary.

We will not take the words with the highest prediction (or the generation of text will be boring), but would like to insert some uncertainties, and let the solution, sometime, to pick-up words with less good prediction.

That is the purpose of the function **sample()**, that will draw randomly a word from our vocabulary.

However, the probability for a word to be drawn will depends directly on its probability to be the next word, thanks to our first bidirectional LSTM Model.

In order to tune this probability, we introduce a "temperature" to smooth or sharpen its value.
 - **if _temperature = 1.0_**, the probability for a word to be drawn is equal to the probability for the word to be the next one in the sequence (output of the owrd prediction model),
 - **if _temperature_ is big (much bigger than 1)**, the range of probabilities is shorten: the probabilities for all words to be the next one is closer to 1. More variety of words will be picked-up from the vocabulary.
 - **if _temperatune_ is small (close to 0)**, small probabilities will be avoided (they will be set closed to 0). Less words will be picked-up from the vocabulary.

In [7]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

The **create_seed()** function is usefull to prepare seed sequences, especially if the number of words in the seed phrase is lower than the espected number for a sequence.

In [8]:
def create_seed(seed_sentences,nb_words_in_seq=20, verbose=False):
    #initiate sentences
    generated = ''
    sentence = []
    
    #fill the sentence with a default word
    for i in range (nb_words_in_seq):
        sentence.append("and")

    seed = seed_sentences.split()
    
    if verbose == True : print("seed: ",seed)

    for i in range(len(sentence)):
        sentence[nb_words_in_seq-i-1]=seed[len(seed)-i-1]
        #print(i, sentence)

    generated += ' '.join(sentence)
    
    if verbose == True : print('Generating text with the following seed: "' + ' '.join(sentence) + '"')

    return [generated, sentence]

the function **generate_phrase()** is used to create the next phrase of a given sentence.

It requires as inputs:
 - the previous sentence,
 - the maximum number of words in the phrase,
 - the temperature of the sample function.
 
If a punctuation word is reached before the maximum number of the words, the function ends.

In [9]:
def generate_phrase(sentence, max_words = 50, nb_words_in_seq=20, temperature=1, verbose = False):
    generated = ""
    words_number = max_words - 1
    ponctuation = [".","?","!",":","…"]
    seq_length = nb_words_in_seq
    #sentence = []
    is_punct = False
    #generate the text
    for i in range(words_number):
        #create the vector
        x = np.zeros((1, seq_length, vocab_size))
        for t, word in enumerate(sentence):
            try:
                x[0, nb_words_in_seq-len(sentence)+t, vocab[word]] = 1.
            except:
                print(t, word, vocab[word])
                print(sentence)
                return
        #print(x.shape)

        #calculate next word
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_word = vocabulary_inv[next_index]
        
        if verbose == True:
            predv = np.array(preds)
            #arr = np.array([1, 3, 2, 4, 5])
            wi = predv.argsort()[-3:][::-1]
            print("potential next words: ", vocabulary_inv[wi[0]], vocabulary_inv[wi[1]], vocabulary_inv[wi[2]])

        #add the next word to the text
        if is_punct == False:
            if next_word in ponctuation:
                is_punct = True
            generated += " " + next_word
            # shift the sentence by one, and and the next word at its end
            sentence = sentence[1:] + [next_word]

    return(generated, sentence)

the function **define_phrases_candidates()** provides a list of potential phrases, for a given previous sentence and a specific temperature.

In [10]:
def define_phrases_candidates(sentence, max_words = 50,\
                              nb_words_in_seq=20, \
                              temperature=1, \
                              nb_candidates_sents=10, \
                              verbose = False):
    phrase_candidate = []
    generated_sentence = ""
    for i in range(nb_candidates_sents):
        generated_sentence, new_sentence = generate_phrase(sentence, \
                                                           max_words = max_words, \
                                                           nb_words_in_seq = nb_words_in_seq, \
                                                           temperature=temperature, \
                                                           verbose = False)
        phrase_candidate.append([generated_sentence, new_sentence])
    
    if verbose == True :
        for phrase in phrase_candidate:
            print("   " , phrase[0])
    return phrase_candidate

# 2. Functions to select the best sentence

the **create_sentences()** function generate a sequence of words (a list) for a given spacy doc item.

It will be used to create a sequence of words from a single phrase.

In [11]:
def create_sentences(doc):
    ponctuation = [".","?","!",":","…"]
    sentences = []
    sent = []
    for word in doc:
        if word.text not in ponctuation:
            if word.text not in ("\n","\n\n",'\u2009','\xa0'):
                sent.append(word.text.lower())
        else:
            sent.append(word.text.lower())
            if len(sent) > 1:
                sentences.append(sent)
            sent=[]
    return sentences

the **generate_training_vector()** function is used to predict the next vectorized-sentence for a given sequence of vectorized-sentences.

In [12]:
def generate_training_vector(sentences_list, verbose = False):
    if verbose == True : print("generate vectors for each sentence...")
    seq = []
    V = []

    for s in sentences_list:
        #infer the vector of the sentence, from the doc2vec model
        v = d2v_model.infer_vector(create_sentences(nlp(s))[0], alpha=0.001, min_alpha=0.001, steps=10000)
    #create the vector array for the model
        V.append(v)
    V_val=np.array(V)
    #expand dimension to fit the entry of the model : that's the training vector
    V_val = np.expand_dims(V_val, axis=0)
    if verbose == True : print("Vectors generated!")
    return V_val

The **select_next_phrase()** function allows us to pick-up the best candidates for the next phrase.

First, it calculates the vector for each candidates.

Then, based on the vector generated by the function **generate_training_vector()**, it performs a cosine similarity with them and pick the one with the biggest similarity.

In [13]:
def select_next_phrase(model, V_val, candidate_list, verbose=False):
    sims_list = []
    
    #calculate prediction
    preds = model.predict(V_val, verbose=0)[0]
    
    #calculate vector for each candidate
    for candidate in candidate_list:
        #calculate vector
        #print("calculate vector for : ", candidate[1])
        V = np.array(d2v_model.infer_vector(candidate[1]))
        #calculate csonie similarity
        sim = scipy.spatial.distance.cosine(V,preds)
        #populate list of similarities
        sims_list.append(sim)
    
    #select index of the biggest similarity
    m = max(sims_list)
    index_max = sims_list.index(m)
    
    if verbose == True :
        print("selected phrase :")
        print("     ", candidate_list[index_max][0])
    return candidate_list[index_max]

# 3. Text generation - workflow

The following function, **generate_paragraph()**, combines all previous functions to generate the text.

With the following parameters:
 - phrase_seed : the sentence seed for the first word prediction. It is a list of words.
 - sentences_seed : the seed sequence of sentences. It is a list of sentences.
 - max_words: the maximum number of words for a new generated sentence.
 - nb_words_in_seq: the number of words to keep as seed for the next word prediction.
 - temperature: the temperature for the word prediction.
 - nb_phrases: the number of phrase (sentence) to generate.
 - nb_candidates_sents: the number of phrase candidates to generate for each new phrase.
 - verbose: verbosity of the script.


In [14]:
def generate_paragraphe(phrase_seed, sentences_seed, \
                        max_words = 50, \
                        nb_words_in_seq=20, \
                        temperature=1, \
                        nb_phrases=30, \
                        nb_candidates_sents=10, \
                        verbose=True):
    
    sentences_list = sentences_seed
    sentence = phrase_seed   
    text = []
    
    for p in range(nb_phrases):
        if verbose == True : print("")
        if verbose == True : print("#############")
        print("phrase ",p+1, "/", nb_phrases)
        if verbose == True : print("#############")       
        if verbose == True:
            print('Sentence to generate phrase : ')
            print("     ", sentence)
            print("")
            print('List of sentences to constrain next phrase : ')
            print("     ", sentences_list)
            print("")
    
        #generate seed training vector
        V_val = generate_training_vector(sentences_list, verbose = verbose)

        #generate phrase candidate
        if verbose == True : print("generate phrases candidates...")
        phrases_candidates = define_phrases_candidates(sentence, \
                                                       max_words = max_words, \
                                                       nb_words_in_seq = nb_words_in_seq, \
                                                       temperature=temperature, \
                                                       nb_candidates_sents=nb_candidates_sents, \
                                                       verbose = verbose)
        
        if verbose == True : print("select next phrase...")
        next_phrase = select_next_phrase(model_sequence, \
                                         V_val,
                                         phrases_candidates, \
                                         verbose=verbose)
        
        print("Next phrase: ",next_phrase[0])
        if verbose == True :
            print("")
            print("Shift phrases in sentences list...")
        for i in range(len(sentences_list)-1):
            sentences_list[i]=sentences_list[i+1]

        sentences_list[len(sentences_list)-1] = next_phrase[0]
        
        if verbose == True:
            print("done.")
            print("new list of sentences :")
            print("     ", sentences_list)     
        sentence = next_phrase[1]
        
        text.append(next_phrase[0])
    
    return text

Now, we can perform the complete text generation workflow.

First, we have to define the sentences in the seed (15 phrases):

In [30]:
s1 = "he tapped meditatively on the table with his thin taper fingers like a man trying to recall a tune . "
s2 = '"in other words a tree sticks in the mud from years end to years end," answered treherne . '
s3 = "you will have a cigar , mr. treherne ? "
s4 = "it seems they have a way of eating things . "
s5 = "squire vane stood up his silver hair flaming in the wind . "
s6 = "She was silent a long time and said at last . "
s7 = "i was born then or woke up or something . "
s8 = "i would greatly appreciate if your brother died ."
s9 = "there are powers there is the spirit of a place there are presences that are not to be put by . "
s10 = "hey you can not do that by simply calling him ."
s11 = "i hate tables because they are always with chairs ."
s12 = "i had walked in this garden like a somnambulist in the sun . "
s13 = "the french minister was arrested shortly after the meeting ."
s14 = "by foreign affairs we do not mean killing ."
s15 = "oh do not fancy i am sentimental and hanker after the good old days . "



In [31]:
from os import listdir
from os.path import isfile, join
import random
file_list = ['G K Chesterton___The Trees of Pride.txt', 'Charles Dickens___The Chimes.txt']
data_dir = "./data/Gutenberg/txt"
num_words = 0
for txt in file_list:
    with open(data_dir+'/'+txt, 'r') as f:
        for line in f:
            words = line.split()
            num_words += len(words)
print("Number of words:")
print(num_words)

Number of words:
55576


We combine them in a list:

In [32]:
sentences_list = []
for i in [s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15]:
    sentences_list.append(unicode(i, 'utf-8'))
print(sentences_list)

[u'he tapped meditatively on the table with his thin taper fingers like a man trying to recall a tune . ', u'"in other words a tree sticks in the mud from years end to years end," answered treherne . ', u'you will have a cigar , mr. treherne ? ', u'it seems they have a way of eating things . ', u'squire vane stood up his silver hair flaming in the wind . ', u'She was silent a long time and said at last . ', u'i was born then or woke up or something . ', u'i would greatly appreciate if your brother died .', u'there are powers there is the spirit of a place there are presences that are not to be put by . ', u'hey you can not do that by simply calling him .', u'i hate tables because they are always with chairs .', u'i had walked in this garden like a somnambulist in the sun . ', u'the french minister was arrested shortly after the meeting .', u'by foreign affairs we do not mean killing .', u'oh do not fancy i am sentimental and hanker after the good old days . ']


We concatenate them in a single phrase and create the seed sentence:

In [33]:
phrase_seed, sentences_seed_one = create_seed(s1 + " " + s2 + " " +\
                                          s3 + " " + s4+ " " + s5 + " " +\
                                          s6 + " " + s7 + " " + s8 + " " +\
                                          s9+ " " + s10 + " " + s11 + " " +\
                                          s12 + " " + s13 + " " + s14+ " " + s15,20)
print(phrase_seed)
print(sentences_seed_one)
sentences_seed = []
for i in sentences_seed_one:
    sentences_seed.append(unicode(i, "utf-8"))
print(sentences_seed)

do not mean killing . oh do not fancy i am sentimental and hanker after the good old days .
['do', 'not', 'mean', 'killing', '.', 'oh', 'do', 'not', 'fancy', 'i', 'am', 'sentimental', 'and', 'hanker', 'after', 'the', 'good', 'old', 'days', '.']
[u'do', u'not', u'mean', u'killing', u'.', u'oh', u'do', u'not', u'fancy', u'i', u'am', u'sentimental', u'and', u'hanker', u'after', u'the', u'good', u'old', u'days', u'.']


In [34]:
for s in sentences_list:
    #print(s)
    print(create_sentences(nlp(s)))

[[u'he', u'tapped', u'meditatively', u'on', u'the', u'table', u'with', u'his', u'thin', u'taper', u'fingers', u'like', u'a', u'man', u'trying', u'to', u'recall', u'a', u'tune', u'.']]
[[u'"', u'in', u'other', u'words', u'a', u'tree', u'sticks', u'in', u'the', u'mud', u'from', u'years', u'end', u'to', u'years', u'end', u',', u'"', u'answered', u'treherne', u'.']]
[[u'you', u'will', u'have', u'a', u'cigar', u',', u'mr', u'.'], [u'treherne', u'?']]
[[u'it', u'seems', u'they', u'have', u'a', u'way', u'of', u'eating', u'things', u'.']]
[[u'squire', u'vane', u'stood', u'up', u'his', u'silver', u'hair', u'flaming', u'in', u'the', u'wind', u'.']]
[[u'she', u'was', u'silent', u'a', u'long', u'time', u'and', u'said', u'at', u'last', u'.']]
[[u'i', u'was', u'born', u'then', u'or', u'woke', u'up', u'or', u'something', u'.']]
[[u'i', u'would', u'greatly', u'appreciate', u'if', u'your', u'brother', u'died', u'.']]
[[u'there', u'are', u'powers', u'there', u'is', u'the', u'spirit', u'of', u'a', u'plac

/home/nyuad/anaconda2/envs/Gabor/lib/python2.7/site-packages/ipykernel_launcher.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  
/home/nyuad/anaconda2/envs/Gabor/lib/python2.7/site-packages/ipykernel_launcher.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  import sys


Run the script to generate the text !

In [35]:
text = generate_paragraphe(sentences_seed, sentences_list, \
                           max_words = 80, \
                           nb_words_in_seq = 30,\
                           temperature=0.201, \
                           nb_phrases=5, \
                           nb_candidates_sents=7, \
                           verbose=False)

phrase  1 / 5


/home/nyuad/anaconda2/envs/Gabor/lib/python2.7/site-packages/ipykernel_launcher.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  
/home/nyuad/anaconda2/envs/Gabor/lib/python2.7/site-packages/ipykernel_launcher.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  import sys
/home/nyuad/anaconda2/envs/Gabor/lib/python2.7/site-packages/ipykernel_launcher.py:34: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


Next phrase:   
 i am , or , i am a of the little to be 
 ; and , and the and 
 of the things , , and a old man , 
 was a man , and a more of , and the of the 
 - which had been a made a .
phrase  2 / 5
Next phrase:   he was 
 a .
phrase  3 / 5
Next phrase:   if you were a heart .
phrase  4 / 5
Next phrase:     i have been 
 a hand a times .
phrase  5 / 5
Next phrase:     , and a hands — .


Then, the new text generated is:

In [36]:
print("generated text: ")
for t in text:
    print(t)

generated text: 
 
 i am , or , i am a of the little to be 
 ; and , and the and 
 of the things , , and a old man , 
 was a man , and a more of , and the of the 
 - which had been a made a .
 he was 
 a .
 if you were a heart .
   i have been 
 a hand a times .
   , and a hands — .
